## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Date
0,0,Verkhozim,RU,52.71,45.42,44.01,99,100,9.73,overcast clouds,2020-11-01 14:33:12
1,1,Presidencia Roque Saenz Pena,AR,-26.79,-60.44,81.00,30,0,4.00,clear sky,2020-11-01 14:33:12
2,2,Kapaa,US,22.08,-159.32,69.80,94,1,6.93,clear sky,2020-11-01 14:33:12
3,3,Bani Walid,LY,31.76,13.99,73.17,38,54,8.34,broken clouds,2020-11-01 14:33:12
4,4,Jamestown,US,42.10,-79.24,46.99,75,90,12.75,light rain,2020-11-01 14:30:34


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Date
1,1,Presidencia Roque Saenz Pena,AR,-26.79,-60.44,81.00,30,0,4.00,clear sky,2020-11-01 14:33:12
21,21,Hithadhoo,MV,-0.60,73.08,83.26,76,100,13.02,light rain,2020-11-01 14:31:42
23,23,Touros,BR,-5.20,-35.46,84.20,66,75,14.99,broken clouds,2020-11-01 14:30:55
25,25,Atuona,PF,-9.80,-139.03,77.85,79,27,15.86,scattered clouds,2020-11-01 14:33:14
27,27,Dwarka,IN,22.24,68.97,83.05,50,45,12.12,scattered clouds,2020-11-01 14:33:14
30,30,Victoria,HK,22.29,114.16,75.99,70,2,10.04,light rain,2020-11-01 14:32:14
31,31,Kaberamaido,UG,1.74,33.16,76.95,70,99,2.53,overcast clouds,2020-11-01 14:33:14
37,37,Iranshahr,IR,27.20,60.68,80.60,5,0,4.70,clear sky,2020-11-01 14:33:14
43,43,Port Moresby,PG,-9.44,147.18,80.60,83,75,5.82,haze,2020-11-01 14:33:15
46,46,Sao Filipe,CV,14.90,-24.50,80.64,68,0,8.05,clear sky,2020-11-01 14:33:15


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
Date                   0
dtype: int64

In [24]:

preferred_cities_df=preferred_cities_df.dropna()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
Date                   0
dtype: int64

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country","Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Presidencia Roque Saenz Pena,AR,81.00,clear sky,-26.79,-60.44,
21,Hithadhoo,MV,83.26,light rain,-0.60,73.08,
23,Touros,BR,84.20,broken clouds,-5.20,-35.46,
25,Atuona,PF,77.85,scattered clouds,-9.80,-139.03,
27,Dwarka,IN,83.05,scattered clouds,22.24,68.97,
30,Victoria,HK,75.99,light rain,22.29,114.16,
31,Kaberamaido,UG,76.95,overcast clouds,1.74,33.16,
37,Iranshahr,IR,80.60,clear sky,27.20,60.68,
43,Port Moresby,PG,80.60,haze,-9.44,147.18,
46,Sao Filipe,CV,80.64,clear sky,14.90,-24.50,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df=hotel_df[(hotel_df["Hotel Name"]!="")]
hotel_clean_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Presidencia Roque Saenz Pena,AR,81.00,clear sky,-26.79,-60.44,Hotel Aconcagua
21,Hithadhoo,MV,83.26,light rain,-0.60,73.08,Scoop Guest House
23,Touros,BR,84.20,broken clouds,-5.20,-35.46,INN NEW HORIZON
25,Atuona,PF,77.85,scattered clouds,-9.80,-139.03,Villa Enata
27,Dwarka,IN,83.05,scattered clouds,22.24,68.97,Hotel Narayan Avezika Comfort
30,Victoria,HK,75.99,light rain,22.29,114.16,Mini Hotel Central
31,Kaberamaido,UG,76.95,overcast clouds,1.74,33.16,"Little Orchard, Kaberamaido"
37,Iranshahr,IR,80.60,clear sky,27.20,60.68,کافی نت دانشگاه
43,Port Moresby,PG,80.60,haze,-9.44,147.18,Lamana Hotel
46,Sao Filipe,CV,80.64,clear sky,14.90,-24.50,Tortuga B&B


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Counry</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_clean_df[["Lat", "Lng"]]
max_temp = hotel_clean_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))